MSBA 6330 Prof Liu

# Spark Lab 8 - Predict bike sharing Using MLlib's pipeline and Gradient-Boosted Trees


**Topics**: *Loading CSV into DataFrame, Droping Columns, Casting column types, split data, plot histogram, plot GroupBy results in a line chart, VectorAssembler, VectorIndexer, GBTRegressor, RegressionEvaluator(RMSE), CrossValidator, ParamGridBuilder, Pipeline, scatter predictions, testing transformers/estimators.* 


This Python notebook demonstrates creating an ML Pipeline to preprocess a dataset, train a Machine Learning model, and make predictions. It is adapted from Databricks's examples.

**Data**: The dataset contains bike rental info from 2011 and 2012 in the Capital bikeshare system, plus additional relevant information such as weather.  This dataset is from Fanaee-T and Gama (2013) and is hosted by the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset).

**Goal**: We want to learn to predict bike rental counts (per hour) from information such as day of the week, weather, season, etc.  Having good predictions of customer demand allows a business or service to prepare and increase supply as needed.

**Approach**: We will use Spark ML Pipelines, which help users piece together parts of a workflow such as feature processing and model training.  We will also demonstrate [model selection (a.k.a. hyperparameter tuning)](http://spark.apache.org/docs/1.6.0/ml-guide.html) using [Cross Validation](http://spark.apache.org/docs/1.6.0/api/python/pyspark.ml.html) in order to fine-tune and improve our ML model.

## Load and understand the data

1\. Download [our data](http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip) and unzip it. We will load the data file `hour.csv` as a dataframe.  We also cache the data so that we only read it from disk once.

In [ ]:
df = 

#### Data description

From the [UCI ML Repository description](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset), we know that the columns have the following meanings.

|field|description|
|--|--|
|instant| record index|
|dteday| date|
|season| season (1:spring, 2:summer, 3:fall, 4:winter)|
|yr| year (0:2011, 1:2012)|
|mnth| month (1 to 12)|
|hr| hour (0 to 23)|
|holiday| whether day is holiday or not|
|weekday| day of the week|
|workingday| if day is neither weekend nor holiday is 1, otherwise is 0.|
|weathersit| 1: Clear, Few clouds, Partly cloudy, Partly cloudy; 2:Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist; 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds; 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog|
|temp| Normalized temperature in Celsius. The values are derived via `(t-t_min)/(t_max-t_min)`, `t_min=-8`, `t_max=+39` (only in hourly scale)|
|atemp| Normalized feeling temperature in Celsius. The values are derived via `(t-t_min)/(t_max-t_min)`, `t_min=-16`, `t_max=+50` (only in hourly scale) |
|hum| Normalized humidity. The values are divided to 100 (max)|
|windspeed| Normalized wind speed. The values are divided to 67 (max)|
|casual | count of casual users|
|registered | count of registered users|
|cnt | count of total rental bikes including both casual and registered|



2\. Now display some basic info about the data
- a few rows of the data
- schema
- number of rows

Note, most columns should be numerical ones. If not, you should adjust your data loading statement.

This dataset is nicely prepared for Machine Learning: values such as weekday are already indexed, and all of the columns except for the date (`dteday`).

## Preprocess data

**label**: We want to predict the count of bike rentals, hence the `cnt` column is our label.

**Features**: We can use the rest as features, except these:
* `casual`, `registered`: The `cnt` column equals the sum of the `casual` + `registered` columns. Unless we are interested in separating `casual` and `registered`, these two are not useful to us. 
* `dteday`:  We will discard it because it is well-represented by the other date-related columns `season`, `yr`, `mnth`, and `weekday`. 
* `instant`: This will not be used for analysis.

3\. Drop `instant`,`dteday`,`casual`,`registered` columns and verify the content and schema

In [ ]:
df2 = 

4\.The Spark ML requires float (python) or double (for scala). Let's cast them to double, then verify the schema

In [ ]:
df3 = 

5\. Split the dataset randomly to keep 70% for training and 30% for testing. Then verify the count of each dataset

## Step 2. Visualize our data

Now that we have preprocessed our features and prepared a training dataset, we can quickly visualize our data to get a sense of whether the features are meaningful.

### Histogram of variables

To do a histogram of values in DataFrame, you must keep in mind that the data volume is large and you may want to collect the data and run the historgram analysis locally. 

- One approach is to use the histogram API implemented in RDD (DF to RDD conversion is required)
- The other approach (less preferred) is to sample the data and collect it to do an approximate local histogram. 

The following function is designed to plot a histogram based on the RDD returned by core spark's histogram API.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

def create_hist(rdd_histogram_data,field):
    """Given an RDD.histogram and field name, plot a histogram bar chart
       example usage: created_hist(rdd_generated_by_spark_histogram_api,"FileName")
    """
    heights = np.array(rdd_histogram_data[1])
    full_bins = rdd_histogram_data[0]
    mid_point_bins = full_bins[:-1]  #mid point of bins except for the last one
    #bar width is calculated as 80% of bin width
    widths = [(i - j)*0.8 for i, j in zip(full_bins[:-1], full_bins[1:])]
    bar = plt.bar(mid_point_bins, heights, width=widths, color='b')
    plt.ylabel('Frequency')
    plt.title('Histogram of '+field)  
    return bar

6\. Choose the `cnt` column and convert it to `RDD[integer]`, then call RDD's `histogram` API, save the result in `hist_rdd`. Then call the above function to plot the resulting histogram RDD. 


The second approach is to 

- take a small enough sample of the data using `Dataframe.sample()`
- then convert it to a Pandas dataframe using for ploting.

7\. Sample the `cnt` column data by 30%, convert it to a Pandas DataFrame `pd_cnt`, then plot the histogram the `cnt` column in `pd_cnt`

### Plot Label and a feature in a line chart

In the below plot, we compare bike rental counts versus hour of the day.  As one might expect, rentals are low during the night, and they peak in the morning (8am) and in the early evening (6pm).  This indicates the `hr` feature is useful and can help us predict our label `cnt`.  On your own, you can try visualizing other features to get a sense of how useful they are in this task.


8\. Compute the hourly count of rental events using Spark SQL, saved the resulting dataframe as `df_cntByHr`

9\. Convert Spark DataFrame `df_cntByHour` to Pandas `pd_cntByHour`, and sort it by values of `hr` (required for plotting)

In [ ]:
pd_cntByHour = 

10\. Plot the `pd_cntByHour` as a line chart

## Step 3. Train a Machine Learning Pipeline

Now that we have understood our data and prepared it as a DataFrame with numeric values, let's learn an ML model to predict bike sharing rentals in the future.  Most ML algorithms expect to predict a single "label" column (`cnt` for our dataset) using a single "features" column of feature vectors.  

We will put together a simple Pipeline with the following stages:
* [`VectorAssembler`](http://spark.apache.org/docs/1.6.0/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler): Assemble the feature columns into a feature vector.
* [`VectorIndexer`](http://spark.apache.org/docs/1.6.0/api/python/pyspark.ml.html#pyspark.ml.feature.VectorIndexer): Identify columns which should be treated as categorical.  This is done heuristically, identifying any column with a small number of distinct values as being categorical.  For us, this will be the `yr` (2 values), `season` (4 values), `holiday` (2 values), `workingday` (2 values), and `weathersit` (4 values).
* [`GBTRegressor`](http://spark.apache.org/docs/1.6.0/api/python/pyspark.ml.html#pyspark.ml.regression.GBTRegressor): This will use the Gradient-Boosted Trees (GBT) algorithm to learn how to predict rental counts from the feature vectors.
* [`RegressionEvaluator`](http://spark.apache.org/docs/1.6.0/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator): This specifies the evaluation metric used for the regression model.
* [`CrossValidator`](http://spark.apache.org/docs/1.6.0/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator): The GBT algorithm has several hyperparameters, and tuning them to our data can improve accuracy.  We will do this tuning using Spark's Cross Validation framework, which automatically tests a grid of hyperparameters and chooses the best.

![Image of Pipeline](http://training.databricks.com/databricks_guide/1-init.png)

First, we define the feature processing stages of the Pipeline:

10\. Create a vector assembler `VectorAssembler` that assembles the feature columns into a feature vector with column name  `rawFeatures`. Name it `va`. Note that all fields in `df3` will be used except for `cnt`


11\. Verify your assembler by transforming `df3` to a new dataframe called `assembled` and show some sample rows from it. 

In [ ]:
## verify your vector assembler


12\. Create a VectorIndexer that indexes all features in `rawFeatures` with <= 4 distinct values. The output column should be called `features`, and the VectorIndex should be called `vi`

13\. Verify your VectorIndexer by transforming `assembled` to a new dataframe called `indexed` and show some sample rows from it. 

14\. Define the GBTRegressor. `GBTRegressor` takes `features` as feature vectors and `cnt` as labels. Save the resulting GBTRegressor as `gbt`. 

15\. We create an evaluator `e` and specify RMSE as the evaluation metric

We wrap the model training stage within a `CrossValidator` stage.  `CrossValidator` knows how to call the GBT algorithm with different hyperparameter settings.  It will train multiple models and choose the best one, based on minimizing some metric. In this example, our metric is **Root Mean Squared Error (RMSE)** as defined above.

16\. Create a parameter grid `paramGrid` that explores two values for GTB's maxDepth parameters: 2 and 5

  - `maxDepth`: max depth of each decision tree in the GBT ensemble
  - `maxIter`: iterations, i.e., number of trees in each GBT ensemble

In this example, we keep these values small.  In practice, to get the highest accuracy, you would likely want to try deeper trees (10 or higher) and more trees in the ensemble (>100).

17\. Create a CrossValidator `cv`, which uses `gbt` as estimator, `e` as evaluator and `paramGrid` as parameter maps. 


Finally, we can tie our feature processing and model training stages together into a single `Pipeline`.

![Image of Pipeline](http://training.databricks.com/databricks_guide/5-pipeline.png)

18\. Create a Pipeline `pipeline` according to the above diagram.

#### Train the Pipeline!

Now that we have set up our workflow, we can train the Pipeline in a single call.  Calling `fit()` will run feature processing, model tuning, and training in a single call.  We get back a fitted Pipeline with the best model found.

***Note***: This next cell can take up to **10 minutes**.  This is because it is training *a lot* of trees:
* For each random sample of data in Cross Validation,
  * For each setting of the hyperparameters,
    * `CrossValidator` is training a separate GBT ensemble which contains many Decision Trees.

19\. Train the pipeline and save the resulting model as `pipelineModel`

## Step 4. Make predictions, and evaluate results

Our final step will be to use our fitted model to make predictions on new data.  We will use our held-out test set, but you could also use this model to make predictions on completely new data.  For example, if we created some features data based on weather predictions for the next week, we could predict bike rentals expected during the next week!

We will also evaluate our predictions.  Computing evaluation metrics is important for understanding the quality of predictions, as well as for comparing models and tuning parameters.

20\. Calling `transform()` on a the `test` dataset to obtain predictions, saving the resulting Dataframe as `predictions`

We expect `predictions` to have a new column `predictions` (as well as intermediate results such as our `rawFeatures` column from previous steps). It is easier to view the results when we limit the columns displayed to:
* `cnt`: the true count of bike rentals
* `prediction`: our predicted count of bike rentals
* feature columns: our original (human-readable) feature columns

21\. Display `cnt`, `prediction`, and original feature columns, limit results to 10 rows

Are these good results?  They are not perfect, but you can see correlation between the counts and predictions.  And there is room to improve---see the next section for ideas to take you further!

Before we continue, we give two tips on understanding results:

**(1) Metrics**: Manually viewing the predictions gives intuition about accuracy, but it can be useful to have a more concrete metric.  Below, we compute an evaluation metric which tells us how well our model makes predictions on all of our data.  In this case (for [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation)), lower is better.  This metric does not mean much on its own, but it can be used to compare different models.  (This is what `CrossValidator` does internally.)

22\. Obtain the RMSE for predictions

**(2) Visualization**: Plotting predictions vs. features can help us make sure that the model "understands" the input features and is using them properly to make predictions.  Below, we can see that the model predictions are correlated with the hour of the day, just like the true labels were.

23\. Plot a scatter plot on a 20% sampel of (`cnt`,`prediction`) pair.

#### Improving our model

There are several ways we could further improve our model:
* **Expert knowledge**: We may not be experts on bike sharing programs, but we know a few things we can use:
  * The count of rentals cannot be negative.  `GBTRegressor` does not know that, but we could threshold the predictions to be `>= 0` post-hoc.
  * The count of rentals is the sum of `registered` and `casual` rentals.  These two counts may have different behavior.  (Frequent cyclists and casual cyclists probably rent bikes for different reasons.)  The best models for this dataset take this into account.  Try training one GBT model for `registered` and one for `casual`, and then add their predictions together to get the full prediction.
* **Better tuning**: To make this notebook run quickly, we only tried a few hyperparameter settings.  To get the most out of our data, we should test more settings.  Start by increasing the number of trees in our GBT model by setting `maxIter=200`; it will take longer to train but can be more accurate.
* **Feature engineering**: We used the basic set of features given to us, but we could potentially improve them.  For example, we may guess that weather is more or less important depending on whether or not it is a workday vs. weekend.  To take advantage of that, we could build a few feature by combining those two base features.  MLlib provides a suite of feature transformers; find out more in the [ML guide](http://spark.apache.org/docs/latest/ml-features.html).

#### Learning more

Check out the other example notebooks in this guide for more ideas about Pipelines, working with datasets, and more. Here are some to start with:
* [Full ML Workflow using Pipelines](https://docs.databricks.com/spark/latest/mllib/index.html)